In [1]:
import os
import pandas as pd
import json
import random
import shutil
import numpy as np
import pathlib
import subprocess
import re
import pickle

In [2]:
source_pose_dir = './raw_data/UBnormal/poses'
source_video_dir = './raw_data/UBnormal/videos'

In [3]:
dst_pose_dir = './STG-NF/data/UBnormal/pose/test'
dst_video_dir = './STG-NF/data/UBnormal/videos'

In [4]:
def get_jsons(directory: str):
    """Function that exctracts the mp4 files from the given directory
    and returns the path to the video and it's annotations
    """
    filelist = [];
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('tracked_person.json'):
                json_file = os.path.join(root,file)
                filelist.append(json_file)
    return filelist

def get_mp4s(directory: str):
    """Function that exctracts the mp4 files from the given directory
    and returns the path to the video and it's annotations
    """
    filelist = [];
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.mp4'):
                json_file = os.path.join(root,file)
                filelist.append(json_file)
    return filelist

def remove_files(directory: str):
    for root, dirs, files in os.walk(directory):
        for file in files:
           file_path = os.path.join(root, file)
           if os.path.isfile(file_path):
               os.remove(file_path)

def run_command(
   command: str,
   cwd: str,
   return_output: bool = False,
   strip_response: bool = True,
) -> list:
   """Prints output of subprocess command in real-time.


   Args:
       command (string): The command to run
       return_output (bool): Return output of command as an array or not.
       strip_response (bool): Whether or not to strip whitespaces from response.


   Returns:
       list|int|None: The output of the command
   """
   output = []
   with subprocess.Popen(
       command,
       cwd=cwd,
       stdout=subprocess.PIPE,
       shell=True,
   ) as process:  # nosec
       while True:
           response = process.stdout.readline()
           if response == b"" and process.poll() is not None:
               break
           if response:
               cleaned_output = response.strip() if strip_response else response
               if return_output:
                   output.append(cleaned_output.decode("UTF-8"))
       command_response = process.poll()
   return output if return_output else command_response

In [5]:
json_files = get_jsons(source_pose_dir)
mp4_files = get_mp4s(source_video_dir)
mp4_dict = {f.split('/')[-1].split('.')[0]: f for f in mp4_files}

In [7]:
def gen_test_set(n_videos):
    test_scenes = []
    while len(test_scenes) < n_videos:
        scene = random.choices(json_files, k=1)[0]
        pose_file_name = scene.split('/')[-1]
        video_name = '_'.join(pose_file_name.split('-')[0].split('_')[0:-3])

        # Validate that the json contains a trajectory
        with open(scene) as f:
            tracking = json.load(f)
            if not tracking:
                continue
        
        # The ground truth shared in the repo of STG-NF contains two formats
        # One is a Numpy array, the other is a text file containing per person if which frame contains invalid poses
        # hence for validating that the poses are abnormal, we're only interrested in the cases where we know on a person level
        # if their pose is abnormal
        # try:
        #     np.load(f'./STG-NF/data/UBnormal/gt/{video_name}_tracks.txt', allow_pickle=True)
        # except FileNotFoundError:
        #     continue
        # except pickle.UnpicklingError:
        #     if scene not in test_scenes:
        #         test_scenes.append(scene)
        annotation_file = f'./STG-NF/data/UBnormal/gt/{video_name}_tracks.txt'
        try:
            mask = np.load(annotation_file, allow_pickle=True)
        except FileNotFoundError:
            continue
        except pickle.UnpicklingError:
            with open(annotation_file) as f:
                annotations = [line.strip() for line in f.readlines()]
            mask = convert_lines_annotation_to_mask(annotations)
            np.save(file=annotation_file, arr=mask, allow_pickle=True)
        test_scenes.append(scene)

    remove_files(dst_pose_dir)
    remove_files(dst_video_dir)
    pathlib.Path(dst_pose_dir).mkdir(parents=True, exist_ok=True)
    pathlib.Path(dst_video_dir).mkdir(parents=True, exist_ok=True)
    for pose_file in test_scenes:
        pose_file_name = pose_file.split('/')[-1]
        video_name = '_'.join(pose_file_name.split('-')[0].split('_')[0:-3])
        video_file = mp4_dict[video_name]
        pose_dest = os.path.join(dst_pose_dir, pose_file_name)
        video_dest = os.path.join(dst_video_dir, f'{video_name}.mp4')
        shutil.copyfile(pose_file, pose_dest)
        shutil.copyfile(video_file, video_dest)
    return test_scenes

def convert_lines_annotation_to_mask(annotations):
    filters = []
    for line in annotations:
        _, start_frame, end_frame = line.split(',')
        # Use float to avoid issues with scientific notation
        filters.append(
                (
                    int(float(start_frame)),
                    int(float(end_frame))
                )
        )
    
    mask = [1] * 451
    for i in range(451):
        for start, end in filters:
            if start <= i <= end:
                mask[i] = 0
    return mask

In [25]:
# TODO: make checkpoint variable
model = 'UBnormal_supervised_79_2'
eval_cmd = f'''python train_eval.py --dataset UBnormal --seg_len 16  --only_test \
--checkpoint /mnt/c/Users/ruben/Documents/VUB/CurrentTrendsOfAI/PoseGraphAnomalyDetection/STG-NF/checkpoints/{model}.tar \
--pose_path_test '/mnt/c/Users/ruben/Documents/VUB/CurrentTrendsOfAI/PoseGraphAnomalyDetection/STG-NF/data/UBnormal/pose/test' \
--pose_path_train '/mnt/c/Users/ruben/Documents/VUB/CurrentTrendsOfAI/PoseGraphAnomalyDetection/STG-NF/data/UBnormal/pose/train' \
--vid_path_train '/mnt/c/Users/ruben/Documents/VUB/CurrentTrendsOfAI/PoseGraphAnomalyDetection/STG-NF/data/UBnormal/videos' \
--vid_path_test '/mnt/c/Users/ruben/Documents/VUB/CurrentTrendsOfAI/PoseGraphAnomalyDetection/STG-NF/data/UBnormal/videos'
'''

In [26]:
# Execute evaluation on 100 random test sets
results = []
for i in range(100):
    print(f"Benchmark {i}")
    test_scenes = gen_test_set(50)
    output = run_command(eval_cmd,
                         cwd='/mnt/c/Users/ruben/Documents/VUB/CurrentTrendsOfAI/PoseGraphAnomalyDetection/STG-NF',
                         return_output=True)
    for line in output:
        if 'Done with' in line:
            split_line = line.split(' ')
            accuracy, samples = split_line[3], split_line[6]
            data_dict = {
                'accuracy': accuracy,
                'n_samples': samples,
                'scenes_used': test_scenes
            }
            results.append(data_dict)
            break

Benchmark 0


100%|██████████| 47/47 [00:01<00:00, 27.27it/s]


Benchmark 1


100%|██████████| 47/47 [00:01<00:00, 24.92it/s]


Benchmark 2


...

100%|██████████| 44/44 [00:01<00:00, 22.99it/s]


Benchmark 98


100%|██████████| 49/49 [00:02<00:00, 18.82it/s]


Benchmark 99


100%|██████████| 48/48 [00:01<00:00, 24.20it/s]


In [24]:
pd.DataFrame(results).to_csv(f'./benchmark_{model}.csv', index=False)